In [1]:
import helper_tools.parser as parser
import importlib
import pandas as pd

importlib.reload(parser)

relation_df, entity_df, docs = parser.redfm_parser("train")
predicate_set_df = relation_df[["predicate", "predicate_uri"]].drop_duplicates()

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 16506.51it/s]


In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_ollama import OllamaEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

ollama_base_url = os.getenv("OLLAMA_BASE_URL")

model = ChatOllama(base_url=ollama_base_url, model="phi4")
embeddings = OllamaEmbeddings(base_url=ollama_base_url, model="nomic-embed-text")

In [8]:
import re

system_prompt = """
You are the Supervisor of a conversation among two agents: entity_extraction_agent and relation_extraction_agent.
The conversation is about extracting information (Closed Information Extraction) from a user-provided text.

You have two options:
1. Call an agent using <goto>agent_name</goto>. Replace agent_name with either entity_extraction_agent or relation_extraction_agent. I.e. <goto>entity_extraction_agent</goto>.
2. Finish the conversation using <goto>FINISH</goto>. Please also output the final result.

Note:
- Do not provide any information yourself, instead use the agents for this.
- The first <goto> tag in your response will be executed.
- Therefore, do include exactly one agent call in your response.
- If you output nothing, this will result in a NoneType Error.
"""

messages = [{"role": "system", "content": system_prompt}]

# Stream the response and accumulate the tokens
full_response = ""
response = model.stream(messages)
for s in response:
    print(s.content, end="")  # Optionally print tokens as they arrive
    full_response += s.content

# Now extract the <goto> tag from the accumulated response
goto_match = re.search(r'<goto>(.*?)</goto>', full_response)
if goto_match:
    goto = goto_match.group(1)
    print(f"\n\nGoto: {goto}")

To proceed with extracting information from the provided text, I will start by identifying and extracting relevant entities within the text. This will allow us to understand what key elements are present before exploring their relationships.

<goto>entity_extraction_agent</goto>  
Please provide the text you would like analyzed for entity extraction.

Goto: entity_extraction_agent
